# Replication - Likelihood Approximation - Table

Here we provide a notebook to replicate the simulation results for the likelihood approximations. 

This produced the table from the main paper.

The notebook replicates the results in:
- /out/simulation/tables/likelihood_approx_MPE.csv
- /out/simulation/tables/likelihood_approx_MAPE.csv

The main script can be found at: 
- /scripts/simulation/tables/likelihood_approx.py



In [1]:
# google colab specific - installing probcox
!pip3 install probcox

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 614kB 17.3MB/s 
  Created wheel for probcox: filename=probcox-0.0.5-cp37-none-any.whl size=5226 sha256=14d304ef158affeb68418472a64a97a27bb7d7aebff50be89a7db30f7382b921
  Stored in directory: /root/.cache/pip/wheels/7a/ed/e5/71cb95c54c96702f1412114114806c74394f421b6cbda6af70
Successfully built probcox


In [2]:
# Modules
# =======================================================================================================================

import os
import sys
import shutil
import subprocess
import tqdm

import numpy as np
import pandas as pd

import torch
from torch.distributions import constraints

import pyro
import pyro.distributions as dist

from pyro.infer import SVI, Trace_ELBO

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import probcox as pcox

dtype = torch.FloatTensor

np.random.seed(90834)
torch.manual_seed(873645)

In [3]:
# Custom function for evaluation
# =======================================================================================================================

# run the approximation 1000 times for a given setting and return MPE/MAPE
def run(surv, pred, batch, est):
    total_obs = surv.shape[0]
    total_events = torch.sum(surv[:, -1] == 1).numpy().tolist()
    sampling_proportion = [total_obs, batch, total_events, None]
    ll = []
    ll2 = []
    while len(ll) <=1000:
        idx = np.unique(np.concatenate((np.random.choice(np.where(surv[:, -1]==1)[0], 2, replace=False), np.random.choice(range(surv.shape[0]), batch-2, replace=False))))
        sampling_proportion[-1] = torch.sum(surv[idx, -1]).numpy().tolist()
        if torch.sum(surv[idx, -1]) > 1:
            e = pcox.CoxPartialLikelihood(pred=pred[idx], sampling_proportion=sampling_proportion).log_prob(surv=surv[idx]).detach().numpy()
            MPE = ((e-est)/est)
            MAPE = np.abs(MPE)
            ll.append(MPE.tolist())
            ll2.append(MAPE.tolist())
    return(np.mean(ll), np.mean(ll2))

In [4]:
# Simulation Settings
# =======================================================================================================================

I = [5000, 10000] # individuals
P = [10, 20] # covariates
C = [0.5, 0.75, 0.95, 0.99] # censorship
B = [64, 128, 256, 512] # batch size


In [5]:
# Simulation 
# =======================================================================================================================

res = np.zeros((16, 4))
res2 = np.zeros((16, 4))
sim_n =[]
ii = 0
jj = 0
for p in P:
    cond = True
    while cond:
        theta = np.random.normal(0, 0.5, (p, 1))
        TVC = pcox.TVC(theta=theta, P_binary=int(p/2), P_continuous=int(p/2), dtype=dtype)
        TVC.make_lambda0(scale=4) # make baselinehazard
        s = np.sum([torch.sum(TVC.sample()[0][:, -1]).numpy() for ii in (range(100))])/100 # check censorship
        if np.logical_and(s>=0.05, s<=0.95): # make sure that we do not have full censorship/events
            cond = False
        theta_ = torch.normal(0, 0.5, (p, 1)).type(dtype) # random parameter initalization for evaluation
    for i in I:
        for c in C:
            # make dataset
            surv, X = TVC.make_dataset(obs=i, fraction_censored=c)

            sim_n.append('I(N): ' + str(i) + '(' + str(surv.shape[0]) + ')' +', P: ' + str(p) + ', C: ' + str(c))

            pred = torch.mm(X, theta_).type(dtype) # linear predictor
            est = pcox.CoxPartialLikelihood(pred=pred, sampling_proportion=None).log_prob(surv=surv).detach().numpy() # evaluate likelihood

            # fit to batch
            for b in tqdm.tqdm(B):
                res[ii, jj], res2[ii, jj] = run(surv=surv, pred=pred, batch=b, est=est)
                jj += 1
            ii += 1
            jj = 0

res = np.round(res, 2)
res2 = np.round(res2, 2)

MPE = pd.DataFrame(np.concatenate((np.asarray(sim_n)[:, None], res.astype(str)), axis=1))
MAPE = pd.DataFrame(np.concatenate((np.asarray(sim_n)[:, None], res2.astype(str)), axis=1))



100%|██████████| 4/4 [01:01<00:00, 15.28s/it]


In [6]:
MPE

,0,1,2,3,4
0,"I(N): 5000(41267), P: 10, C: 0.5",0.02,0.01,0.0,0.0
1,"I(N): 5000(39895), P: 10, C: 0.75",0.01,0.0,0.0,-0.0
2,"I(N): 5000(38961), P: 10, C: 0.95",0.02,0.01,-0.0,0.0
3,"I(N): 5000(39181), P: 10, C: 0.99",0.01,0.0,-0.0,0.0
4,"I(N): 10000(82470), P: 10, C: 0.5",0.01,0.0,0.0,-0.0
5,"I(N): 10000(80139), P: 10, C: 0.75",0.01,0.0,0.0,0.0
6,"I(N): 10000(77624), P: 10, C: 0.95",0.01,0.0,0.0,-0.0
7,"I(N): 10000(77930), P: 10, C: 0.99",0.01,0.0,-0.0,-0.0
8,"I(N): 5000(41965), P: 20, C: 0.5",-0.06,-0.05,-0.03,-0.02
9,"I(N): 5000(40103), P: 20, C: 0.75",-0.06,-0.04,-0.03,-0.02


In [7]:
MAPE

,0,1,2,3,4
0,"I(N): 5000(41267), P: 10, C: 0.5",0.05,0.04,0.03,0.02
1,"I(N): 5000(39895), P: 10, C: 0.75",0.06,0.05,0.04,0.03
2,"I(N): 5000(38961), P: 10, C: 0.95",0.07,0.07,0.06,0.05
3,"I(N): 5000(39181), P: 10, C: 0.99",0.07,0.07,0.07,0.07
4,"I(N): 10000(82470), P: 10, C: 0.5",0.05,0.04,0.03,0.02
5,"I(N): 10000(80139), P: 10, C: 0.75",0.05,0.05,0.04,0.03
6,"I(N): 10000(77624), P: 10, C: 0.95",0.07,0.07,0.06,0.05
7,"I(N): 10000(77930), P: 10, C: 0.99",0.07,0.07,0.07,0.06
8,"I(N): 5000(41965), P: 20, C: 0.5",0.09,0.07,0.05,0.04
9,"I(N): 5000(40103), P: 20, C: 0.75",0.1,0.08,0.06,0.04
